In [1]:
import pandas as pd
import operator
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

### 데이터 불러오기

In [2]:
#고객, 판권, 다운로드
customer = pd.read_csv('./Data_OTT/movie_customer.csv', engine = 'python', encoding = 'cp949')
inventory = pd.read_csv('./Data_OTT/movie_inventory1.csv', engine = 'python', encoding = 'cp949')
down = pd.read_csv('./Data_OTT/movie_down.csv', engine='python', encoding='cp949')

inventory.drop('Unnamed: 0', axis = 1, inplace = True)

display(inventory.head())
display(down.head())
display(customer.head())

,item_id,contract_year,movie_id,title,price,release_year,release_date,runtime,mpa_rating,mpa_rating_origin,...,director,Genre_1,Genre_2,Genre_3,actor_1,actor_2,actor_3,contract_price,studio_score,price_class
0,I-1001,2015,tt0121766,Star Wars: Episode III - Revenge of the Sith,5500.0,2005,05-19-2005,140,PG-13,PG-13,...,George Lucas,Action,Adventure,Fantasy,Ewan McGregor,Natalie Portman,Hayden Christensen,2700,10,10
1,I-1002,2015,tt0330373,Harry Potter and the Goblet of Fire,4500.0,2005,11-18-2005,157,PG-13,PG-13,...,Mike Newell,Adventure,Family,Fantasy,Eric Sykes,Timothy Spall,David Tennant,2700,10,10
2,I-1003,2015,tt0363771,"The Chronicles of Narnia: The Lion, the Witc",5500.0,2005,12-09-2005,143,PG,PG,...,Andrew Adamson,Adventure,Family,Fantasy,Georgie Henley,Skandar Keynes,William Moseley,2295,10,1
3,I-1004,2015,tt0383574,Pirates of the Caribbean: Dead Man's Chest,5500.0,2006,07-07-2006,151,PG-13,PG-13,...,Gore Verbinski,Action,Adventure,Fantasy,Johnny Depp,Orlando Bloom,Keira Knightley,2295,10,1
4,I-1005,2015,tt0413300,Spider-Man 3,4000.0,2007,05-04-2007,139,PG-13,PG-13,...,Sam Raimi,Action,Adventure,Sci-Fi,Tobey Maguire,Kirsten Dunst,James Franco,2295,10,1


,customer_id,item_id,down_date,down_year,down_month,down_weekday
0,C5001,I-1038,2015-01-08,2015,2015M01,Thu
1,C5001,I-1003,2015-01-17,2015,2015M01,Sat
2,C5001,I-1017,2015-01-17,2015,2015M01,Sat
3,C5001,I-1028,2015-01-19,2015,2015M01,Mon
4,C5001,I-1036,2015-01-20,2015,2015M01,Tue


,customer_id,gender,age,age_group,married,kids_under12,area,subs_start_year
0,C5001,F,51,50대,M,N,용산구,2015
1,C5002,M,22,20대,S,N,강남구,2015
2,C5003,F,33,30대,M,Y,서대문구,2015
3,C5004,F,24,20대,M,N,서대문구,2015
4,C5005,F,35,30대,M,N,마포구,2015


In [3]:
# 다운로드, 판권 데이터 합치기
data = pd.merge(down, inventory, on = 'item_id')

In [4]:
# 다운로드, 판권, 고객 데이터 합치기
data = pd.merge(data, customer, on = 'customer_id')
data.head()

,customer_id,item_id,down_date,down_year,down_month,down_weekday,contract_year,movie_id,title,price,...,contract_price,studio_score,price_class,gender,age,age_group,married,kids_under12,area,subs_start_year
0,C5001,I-1038,2015-01-08,2015,2015M01,Thu,2015,tt1951264,The Hunger Games: Catching Fire,1000.0,...,2600,6,10,F,51,50대,M,N,용산구,2015
1,C5001,I-1038,2016-07-31,2016,2016M07,Sun,2015,tt1951264,The Hunger Games: Catching Fire,1000.0,...,2600,6,10,F,51,50대,M,N,용산구,2015
2,C5001,I-1003,2015-01-17,2015,2015M01,Sat,2015,tt0363771,"The Chronicles of Narnia: The Lion, the Witc",5500.0,...,2295,10,1,F,51,50대,M,N,용산구,2015
3,C5001,I-1003,2017-07-13,2017,2017M07,Thu,2015,tt0363771,"The Chronicles of Narnia: The Lion, the Witc",5500.0,...,2295,10,1,F,51,50대,M,N,용산구,2015
4,C5001,I-1017,2015-01-17,2015,2015M01,Sat,2015,tt1055369,Transformers: Revenge of the Fallen,5000.0,...,2145,4,1,F,51,50대,M,N,용산구,2015


In [56]:
# display(inventory[['income_usa']])
# max(inventory['income_usa'])

,income_usa
0,380270577
1,290417905
2,291710957
3,423315812
4,336530303
...,...
101,216648740
102,220159104
103,216428042
104,201091711


760507625

In [74]:
# for i in range(len(inventory['income_usa'])):
#     inventory['income_usa'][i] = max(inventory['income_usa']) - inventory['income_usa'][i]
# display(inventory[['income_usa']])

def weight_rating(x):
    v = x['votes']
    r = x['imdb_score']
    return (v/(v+m)*r) + (m/(m+v)*C)

def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = down_data_group.iloc[movie_indices]
    vote_counts = movies[movies['votes'].notnull()]['votes'].astype('int')
    vote_averages = movies[movies['imdb_score'].notnull()]['imdb_score'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['votes'] >= m) & (movies['votes'].notnull()) & (movies['imdb_score'].notnull())]
    qualified['votes'] = qualified['votes'].astype('int')
    qualified['imdb_score'] = qualified['imdb_score'].astype('int')
    qualified['wr'] = qualified.apply(weight_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

## 기존 사용자를 위한 하이브리드 알고리즘

In [15]:
def find_customer():
    id = input("회원번호를 입력하세요: ")
    
    #고객 id를 입력받고 성별, 연령대, 혼인여부, 12세이하 자녀 여부를 변수에 저장
    id_gender = customer[customer['customer_id'] == id]['gender'].reset_index(drop = True)[0]
    id_age_group = customer[customer['customer_id'] == id]['age_group'].reset_index(drop = True)[0]
    id_married = customer[customer['customer_id'] == id]['married'].reset_index(drop = True)[0]
    id_kids_under12 = customer[customer['customer_id'] == id]['kids_under12'].reset_index(drop = True)[0]
    
    print("")
    print('회원번호 :', id)
    print('성별 :', id_gender)
    print('연령대 :', id_age_group)
    print('결혼 여부 :', id_married)
    print('12세 이하 자녀 유무 :', id_kids_under12)
    
    id_down_data = data[data['customer_id'] == id]
    genre_counter = Counter(id_down_data['Genre_1']) + Counter(id_down_data['Genre_2']) + Counter(id_down_data['Genre_3'])
    
    id_genre_list = []
    
    id_genre_list.append(sorted(genre_counter.items(), key = operator.itemgetter(1), reverse = True)[0][0])
    id_genre_list.append(sorted(genre_counter.items(), key = operator.itemgetter(1), reverse = True)[1][0])
    id_genre_list.append(sorted(genre_counter.items(), key = operator.itemgetter(1), reverse = True)[2][0])
    
    print('선호 장르 :', id_genre_list[0], id_genre_list[1], id_genre_list[2])
    
    #조건을 만족하는 다운로드 자료들을 down_data에 저장
    down_data = data[(data['gender'] == id_gender) & 
                     (data['age_group'] == id_age_group) &
                    (data['married'] == id_married) & 
                     (data['kids_under12'] == id_kids_under12)]
    
    down_data = down_data[(down_data['Genre_1'].isin(id_genre_list)) | 
                          (down_data['Genre_2'].isin(id_genre_list)) | 
                          (down_data['Genre_3'].isin(id_genre_list))]
    
    #입력받은 id의 고객이 본 영화 id
    id_movie_list = list(pd.unique(down_data[down_data['customer_id'] == id]['item_id']))
    #입력받은 id 고객 + 조건을 만족하는 다른 사람들의 영화 id
    other_movie_list = list(pd.unique(down_data['item_id']))
     
    #입력받은 고객에게 추천하기 위해 other_movie_list에 있는 영화들만 판권 데이터와 병합
    movie_data = pd.DataFrame({'item_id' : other_movie_list})
    down_data_group = pd.merge(movie_data, inventory, on = 'item_id')
    
#     for i in range(len(down_data_group['reviews_users'])):
#         down_data_group['reviews_users'][i] = max(down_data_group['reviews_users']) - down_data_group['reviews_users'][i]
    
#     for i in range(len(down_data_group['income_usa'])):
#         down_data_group['income_usa'][i] = max(down_data_group['income_usa']) - down_data_group['income_usa'][i]
    
    
    display(down_data_group[['title', 'Genre_1', 'Genre_2', 'Genre_3', 'actor_1', 'director']])
    
    #전체 리뷰수와 평균 평점으로 가중치 생성
    m = down_data_group['reviews_users'].quantile(0.96)
    C = down_data_group['income_usa'].mean()
    
    #m 값 이상의 데이터들만 가져옴
    qualified = down_data_group[(down_data_group['reviews_users'].notnull()) & 
                                (down_data_group['income_usa'].notnull()) & 
               (down_data_group['reviews_users'] >= m)]

    qualified['reviews_users'] = qualified['reviews_users'].astype(int)
    qualified['income_usa'] = qualified['income_usa'].astype(int)

    #점수를 만드는 함수
    def weight_rating(x):
        v = x['reviews_users']
        r = x['income_usa']
        return (v/(v+m)*r) + (m/(m+v)*C)
    
    #wr = 점수
    qualified['wr'] = qualified.apply(weight_rating, axis=1)
    qualified = qualified.sort_values(by='wr', ascending=True).head(250)
    
    #감독, 배우1, 장르1,2,3 을 list에 각각 담음
    director_list = list(down_data_group['director'])
    actor_list = list(down_data_group['actor_1'])
    genre_list1 = list(down_data_group['Genre_1'])
    genre_list2 = list(down_data_group['Genre_2'])
    genre_list3 = list(down_data_group['Genre_3'])
    
    #한 영화의 장르1,2,3 을 담기 위한 리스트 생성
    #ex. [Action Adventure Sci-Fi, Drama, Comedy...]
    genre_list = []
    
    #장르2 가 없으면 장르1만 추가
    #장르3이 없으면 장르1 + " " + 장르2 로 추가
    #장르가 다 있으면 장르1 + " " + 장르2 + " " + 장르3 으로 추가
    for i in range(len(down_data_group['Genre_1'])):
        if down_data_group['Genre_2'].isnull()[i] == True:
            gen = genre_list1[i]
            genre_list.append(gen)
        elif down_data_group['Genre_3'].isnull()[i] == True:
            gen = genre_list1[i] + str(" ") + genre_list2[i]
            genre_list.append(gen)
        else:
            gen = genre_list1[i] + str(" ") + genre_list2[i] + str(" ") + genre_list3[i]
            genre_list.append(gen)
    
    #요인들을 한 열에 넣기 위한 리스트
    soup_list = []
    
    #감독 이름 공백제거 + " " + 배우 이름 공백제거 + " " +
    #장르 추가
    for i in range(len(director_list)):
        soup = director_list[i].replace(" ", "") + str(" ") + \
        actor_list[i].replace(" ", "") + str(" ")
        for j in range(len(genre_list[i].split())):
            if j == len(genre_list[i].split()) - 1:
                soup += genre_list[i].split()[j]
            else:
                soup += genre_list[i].split()[j] + str(" ")
        soup_list.append(soup)
    
    #soup열을 생성하고 추가
    down_data_group['soup'] = soup_list
    
    #1개부터 3개 조합까지 고려해서 단어 백터 생성
    count = CountVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,
                       stop_words='english')
    count_matrix = count.fit_transform(down_data_group['soup'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    #index, 영화제목의 indices
    titles = down_data_group['title']
    indices = pd.Series(down_data_group.index, index = down_data_group['title'])

    id_movie_data = down[down['item_id'].isin(id_movie_list)]
    id_movie_data_id = id_movie_data.groupby('item_id').count().sort_values(by = 'customer_id', ascending = False).index[0]
    id_movie_title = inventory[inventory['item_id'] == id_movie_data_id]['title'].reset_index(drop = True)[0]

    idx = indices[id_movie_title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = down_data_group.iloc[movie_indices]
    vote_counts = movies[movies['reviews_users'].notnull()]['reviews_users'].astype('int')
    vote_averages = movies[movies['income_usa'].notnull()]['income_usa'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['reviews_users'] >= m) & (movies['reviews_users'].notnull()) & (movies['income_usa'].notnull())]
    qualified['reviews_users'] = qualified['reviews_users'].astype('int')
    qualified['income_usa'] = qualified['income_usa'].astype('int')
    qualified['wr'] = qualified.apply(weight_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=True).head(10)
    
    for i in id_movie_list:
        other_movie_list.remove(i)
    
    qualified = qualified[qualified['item_id'].isin(other_movie_list)]
    
    return qualified

In [22]:
result = find_customer()
# result[['title', 'Genre_1', 'Genre_2', 'Genre_3', 'actor_1', 'director']].reset_index(drop = True)
result[['title', 'Genre_1', 'Genre_2', 'Genre_3', 'actor_1', 'director', 'soup', 'wr', 'reviews_users', 'income_usa']].reset_index(drop = True)

회원번호를 입력하세요: C7122

회원번호 : C7122
성별 : M
연령대 : 20대
결혼 여부 : S
12세 이하 자녀 유무 : N
선호 장르 : Adventure Action Sci-Fi


,title,Genre_1,Genre_2,Genre_3,actor_1,director
0,The Hunger Games: Catching Fire,Action,Adventure,Mystery,Jennifer Lawrence,Francis Lawrence
1,I Am Legend,Adventure,Drama,Sci-Fi,Will Smith,Francis Lawrence
2,The Dark Knight Rises,Action,Adventure,NaN,Christian Bale,Christopher Nolan
3,Shrek the Third,Animation,Adventure,Comedy,Mike Myers,Chris Miller
4,Captain America: The Winter Soldier,Action,Adventure,Sci-Fi,Chris Evans,Anthony Russo
...,...,...,...,...,...,...
74,Captain America: Civil War,Action,Adventure,Sci-Fi,Chris Evans,Anthony Russo
75,Gravity,Drama,Sci-Fi,Thriller,Sandra Bullock,Alfonso Cuarn
76,The Hobbit: An Unexpected Journey,Adventure,Family,Fantasy,Ian McKellen,Peter Jackson
77,Pirates of the Caribbean: Dead Man's Chest,Action,Adventure,Fantasy,Johnny Depp,Gore Verbinski


<ipython-input-15-0e34f1881d5a>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['reviews_users'] = qualified['reviews_users'].astype(int)
<ipython-input-15-0e34f1881d5a>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['income_usa'] = qualified['income_usa'].astype(int)
<ipython-input-15-0e34f1881d5a>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,title,Genre_1,Genre_2,Genre_3,actor_1,director,soup,wr,reviews_users,income_usa
0,Mad Max: Fury Road,Action,Adventure,Sci-Fi,Tom Hardy,George Miller,GeorgeMiller TomHardy Action Adventure Sci-Fi,2.232704e+08,1886,154058340
1,Star Trek,Action,Adventure,Sci-Fi,Chris Pine,J.J. Abrams,J.J.Abrams ChrisPine Action Adventure Sci-Fi,2.831581e+08,1620,257730019
2,I Am Legend,Adventure,Drama,Sci-Fi,Will Smith,Francis Lawrence,FrancisLawrence WillSmith Adventure Drama Sci-Fi,2.835440e+08,1496,256393010
3,Gravity,Drama,Sci-Fi,Thriller,Sandra Bullock,Alfonso Cuarn,AlfonsoCuarn SandraBullock Drama Sci-Fi Thriller,2.896427e+08,2027,274092705
4,Inception,Action,Adventure,Sci-Fi,Leonardo DiCaprio,Christopher Nolan,ChristopherNolan LeonardoDiCaprio Action Adven...,2.980811e+08,3439,292576195
5,Man of Steel,Action,Adventure,Sci-Fi,Henry Cavill,Zack Snyder,ZackSnyder HenryCavill Action Adventure Sci-Fi,2.980892e+08,2686,291045518
6,Transformers,Action,Adventure,Sci-Fi,Shia LaBeouf,Michael Bay,MichaelBay ShiaLaBeouf Action Adventure Sci-Fi,3.154516e+08,1840,319246193
7,Spider-Man 3,Action,Adventure,Sci-Fi,Tobey Maguire,Sam Raimi,SamRaimi TobeyMaguire Action Adventure Sci-Fi,3.255599e+08,2011,336530303
8,The Hunger Games,Action,Adventure,Sci-Fi,Stanley Tucci,Gary Ross,GaryRoss StanleyTucci Action Adventure Sci-Fi,3.664791e+08,2001,408010692


## 신규 사용자를 위한 하이브리드 알고리즘

In [33]:
def new_customer():
    #고객 id를 입력받고 성별, 연령대, 혼인여부, 12세이하 자녀 여부를 변수에 저장
    
    name = input("이름: ")
    gender = input("성별(M/F): ")
    age = int(input("나이: "))
    married = input("결혼유무(M/S): ")
    kids_under12 = input("12세이하 자녀유무(Y/N): ")
    genre = input("1, 2, 3순위 선호 장르를 입력하세요(하나 이상 입력): ")
    
    id_gender = gender
    id_age_group = str(int(age / 10) * 10) + str('대')
    id_married = married
    id_kids_under12 = kids_under12
    
    print("")
    print('이름 :', name)
    print('성별 :', id_gender)
    print('연령대 :', id_age_group)
    print('결혼여부 :', id_married)
    print('12세 이하 자녀 유무 :', id_kids_under12)
    print('선호하는 장르', genre.split())
    print("")
    
    id_genre_list = genre.split()
    for i in range(len(id_genre_list)):
        if id_genre_list[i].lower() != 'sci-fi':
            id_genre_list[i] = str(id_genre_list[i][0].upper()) + str(id_genre_list[i][1:])
        else:
            id_genre_list[i] = str(id_genre_list[i][0].upper()) + str(id_genre_list[i][1:4]) + str(id_genre_list[i][4].upper()) + str(id_genre_list[i][5])

    #조건을 만족하는 다운로드 자료들을 down_data에 저장
    down_data = data[(data['gender'] == id_gender) & 
                     (data['age_group'] == id_age_group) &
                    (data['married'] == id_married) & 
                     (data['kids_under12'] == id_kids_under12)]
    
    down_data = down_data[(down_data['Genre_1'].isin(id_genre_list)) | 
                          (down_data['Genre_2'].isin(id_genre_list)) | 
                          (down_data['Genre_3'].isin(id_genre_list))]
    
    
    display(down_data[['title', 'Genre_1', 'Genre_2', 'Genre_3', 'actor_1', 'director']].reset_index(drop = True))
    #입력받은 id의 고객이 본 영화 id
    #id_movie_list = list(pd.unique(down_data[down_data['customer_id'] == id]['item_id']))
    #입력받은 id 고객 + 조건을 만족하는 다른 사람들의 영화 id
    other_movie_list = list(pd.unique(down_data['item_id']))
     
    #입력받은 고객에게 추천하기 위해 other_movie_list에 있는 영화들만 판권 데이터와 병합
    movie_data = pd.DataFrame({'item_id' : other_movie_list})
    down_data_group = pd.merge(movie_data, inventory, on = 'item_id')

    #전체 리뷰수와 평균 평점으로 가중치 생성
    m = down_data_group['votes'].quantile(0.96)
    C = down_data_group['imdb_score'].mean()
    
    #m 값 이상의 데이터들만 가져옴
    qualified = down_data_group[(down_data_group['votes'].notnull()) & 
                                (down_data_group['imdb_score'].notnull()) & 
               (down_data_group['votes'] >= m)]

    qualified['votes'] = qualified['votes'].astype(int)
    qualified['imdb_score'] = qualified['imdb_score'].astype(int)

    #점수를 만드는 함수
    def weight_rating(x):
        v = x['votes']
        r = x['imdb_score']
        return (v/(v+m)*r) + (m/(m+v)*C)
    
    #wr = 점수
    qualified['wr'] = qualified.apply(weight_rating, axis=1)
    qualified = qualified.sort_values(by='wr', ascending=False).head(250)
 
    #감독, 배우1, 장르1,2,3 을 list에 각각 담음
    director_list = list(down_data_group['director'])
    actor_list = list(down_data_group['actor_1'])
    genre_list1 = list(down_data_group['Genre_1'])
    genre_list2 = list(down_data_group['Genre_2'])
    genre_list3 = list(down_data_group['Genre_3'])
    
    #한 영화의 장르1,2,3 을 담기 위한 리스트 생성
    #ex. [Action Adventure Sci-Fi, Drama, Comedy...]
    genre_list = []
    
    #장르2 가 없으면 장르1만 추가
    #장르3이 없으면 장르1 + " " + 장르2 로 추가
    #장르가 다 있으면 장르1 + " " + 장르2 + " " + 장르3 으로 추가
    for i in range(len(down_data_group['Genre_1'])):
        if down_data_group['Genre_2'].isnull()[i] == True:
            gen = genre_list1[i]
            genre_list.append(gen)
        elif down_data_group['Genre_3'].isnull()[i] == True:
            gen = genre_list1[i] + str(" ") + genre_list2[i]
            genre_list.append(gen)
        else:
            gen = genre_list1[i] + str(" ") + genre_list2[i] + str(" ") + genre_list3[i]
            genre_list.append(gen)

    #요인들을 한 열에 넣기 위한 리스트
    soup_list = []
    
    #감독 이름 공백제거 + " " + 배우 이름 공백제거 + " " +
    #장르 추가
    for i in range(len(director_list)):
        soup = director_list[i].replace(" ", "") + str(" ") + \
        actor_list[i].replace(" ", "") + str(" ")
        for j in range(len(genre_list[i].split())):
            if j == len(genre_list[i].split()) - 1:
                soup += genre_list[i].split()[j]
            else:
                soup += genre_list[i].split()[j] + str(" ")
        soup_list.append(soup)
    
    #soup열을 생성하고 추가
    down_data_group['soup'] = soup_list

    #1개부터 3개 조합까지 고려해서 단어 백터 생성
    count = CountVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,
                       stop_words='english')
    count_matrix = count.fit_transform(down_data_group['soup'])
    
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    display(cosine_sim)
    
    #index, 영화제목의 indices
    titles = down_data_group['title']
    indices = pd.Series(down_data_group.index, index = down_data_group['title'])

    id_movie_data = down[down['item_id'].isin(other_movie_list)]
    id_movie_data_id = id_movie_data.groupby('item_id').count().sort_values(by = 'customer_id', ascending = False).index[0]
    id_movie_title = inventory[inventory['item_id'] == id_movie_data_id]['title'].reset_index(drop = True)[0]
    idx = indices[id_movie_title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = down_data_group.iloc[movie_indices]
    vote_counts = movies[movies['votes'].notnull()]['votes'].astype('int')
    vote_averages = movies[movies['imdb_score'].notnull()]['imdb_score'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['votes'] >= m) & (movies['votes'].notnull()) & (movies['imdb_score'].notnull())]
    qualified['votes'] = qualified['votes'].astype('int')
    qualified['imdb_score'] = qualified['imdb_score'].astype('int')
    qualified['wr'] = qualified.apply(weight_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    
#     for i in id_movie_list:
#         other_movie_list.remove(i)
    
#     qualified = qualified[qualified['item_id'].isin(other_movie_list)]
    
    return qualified

In [36]:
# new_customer()
result2 = new_customer()
result2[['title', 'Genre_1', 'Genre_2', 'Genre_3', 'actor_1', 'director']].reset_index()

이름: 문찬국
성별(M/F): M
나이: 27
결혼유무(M/S): S
12세이하 자녀유무(Y/N): N
1, 2, 3순위 선호 장르를 입력하세요(하나 이상 입력): drama history

이름 : 문찬국
성별 : M
연령대 : 20대
결혼여부 : S
12세 이하 자녀 유무 : N
선호하는 장르 ['drama', 'history']



,title,Genre_1,Genre_2,Genre_3,actor_1,director
0,I Am Legend,Adventure,Drama,Sci-Fi,Will Smith,Francis Lawrence
1,I Am Legend,Adventure,Drama,Sci-Fi,Will Smith,Francis Lawrence
2,I Am Legend,Adventure,Drama,Sci-Fi,Will Smith,Francis Lawrence
3,American Sniper,Action,Biography,Drama,Bradley Cooper,Clint Eastwood
4,The Dark Knight,Action,Crime,Drama,Christian Bale,Christopher Nolan
...,...,...,...,...,...,...
1247,Dunkirk,Action,Drama,History,Fionn Whitehead,Christopher Nolan
1248,Straight Outta Compton,Biography,Drama,History,O'Shea Jackson Jr.,F. Gary Gray
1249,Dunkirk,Action,Drama,History,Fionn Whitehead,Christopher Nolan
1250,Dunkirk,Action,Drama,History,Fionn Whitehead,Christopher Nolan


<ipython-input-33-e1477890bb80>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['votes'] = qualified['votes'].astype(int)
<ipython-input-33-e1477890bb80>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['imdb_score'] = qualified['imdb_score'].astype(int)
<ipython-input-33-e1477890bb80>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

array([[1.        , 0.0745356 , 0.0745356 , 0.0745356 , 0.06666667,
        0.0745356 , 0.0745356 , 0.2236068 , 0.0745356 , 0.06666667,
        0.2236068 , 0.2236068 , 0.6       , 0.4       ],
       [0.0745356 , 1.        , 0.16666667, 0.25      , 0.0745356 ,
        0.16666667, 0.08333333, 0.16666667, 0.25      , 0.2236068 ,
        0.08333333, 0.08333333, 0.0745356 , 0.0745356 ],
       [0.0745356 , 0.16666667, 1.        , 0.08333333, 0.0745356 ,
        0.25      , 0.08333333, 0.16666667, 0.08333333, 0.0745356 ,
        0.08333333, 0.08333333, 0.0745356 , 0.0745356 ],
       [0.0745356 , 0.25      , 0.08333333, 1.        , 0.0745356 ,
        0.08333333, 0.08333333, 0.08333333, 0.25      , 0.2236068 ,
        0.08333333, 0.08333333, 0.0745356 , 0.1490712 ],
       [0.06666667, 0.0745356 , 0.0745356 , 0.0745356 , 1.        ,
        0.0745356 , 0.2236068 , 0.0745356 , 0.0745356 , 0.06666667,
        0.0745356 , 0.0745356 , 0.06666667, 0.13333333],
       [0.0745356 , 0.16666667, 0.2

<ipython-input-33-e1477890bb80>:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['votes'] = qualified['votes'].astype('int')
<ipython-input-33-e1477890bb80>:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualified['imdb_score'] = qualified['imdb_score'].astype('int')
<ipython-input-33-e1477890bb80>:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,index,title,Genre_1,Genre_2,Genre_3,actor_1,director
0,2,The Dark Knight,Action,Crime,Drama,Christian Bale,Christopher Nolan
1,13,Gravity,Drama,Sci-Fi,Thriller,Sandra Bullock,Alfonso Cuarn
2,0,I Am Legend,Adventure,Drama,Sci-Fi,Will Smith,Francis Lawrence
3,5,Dunkirk,Action,Drama,History,Fionn Whitehead,Christopher Nolan
4,1,American Sniper,Action,Biography,Drama,Bradley Cooper,Clint Eastwood


data[data['Genre_3'].isin(['History','Drama']) | data['Genre_2'].isin(['History','Drama']) | data['Genre_1'].isin(['History','Drama'])][['title', 'Genre_1', 'Genre_2', 'Genre_3']].head(50)